In [2]:
import pandas as pd
import numpy as np
import json
import re

#https://bl.ocks.org/rofrischmann/0de01de85296591eb45a1dde2040c5a1

In [3]:
details = pd.read_csv("../Data/extra_details_complete.csv",index_col=0)

# Sort database by descending metascore
details = details.sort_values('metascore',ascending=False)

np.unique(details.loc[:,'number of online players'].replace(np.nan, ''))

array(['', ' ', '10 Players Online',
       '10 Players Online No Online Multiplayer',
       '10 Players Online Online Multiplayer',
       '10 Players Online Up to 10 Players', '12 Players Online',
       '12 Players Online 2 Players Up to 12 Players',
       '12 Players Online Online Multiplayer',
       '12 Players Online Up to 10 Players',
       '12 Players Online Up to 12 Players', '14 Players Online',
       '14 Players Online Up to 14 Players', '16 Players Online',
       '16 Players Online No Online Multiplayer',
       '16 Players Online Up to 12 Players',
       '16 Players Online Up to 16 Players',
       '16 Players Online Up to 18 Players', '2 Players',
       '2 Players 2 Players Online', '2 Players 4 Players Online',
       '2 Players 8 Players Online', '2 Players Online',
       '2 Players Online Multiplayer',
       '2 Players Online No Online Multiplayer',
       '2 Players Online Up to 12 Players', '24 Players Online',
       '24 Players Online No Online Multiplaye

## Find specific game and all its neigbours

In [37]:
# Filter database first for one platform only
ps3_details = details.loc[details['console']=='PS3']

## Functions for finding root, finding possible neighbours and making nodes

In [95]:
def GetGameFeatures(game, console_dataframe):
    '''Takes a console specific dataframe and game name as input. Returns the relevant features of that game'''
    game_row = console_dataframe.loc[console_dataframe['name'] == game]
    franchise = list(game_row['franchise'])[0]
    developers = list(game_row['developer'])[0]
    publishers = list(game_row['publisher'])[0]
    genres = list(game_row['genre(s)'])[0]
    online_mp = list(game_row['number of online players'])[0]
    offline_mp = list(game_row['number of players'])[0]
    
    if type(franchise) == str:
        franchise = franchise.split(',')
    else: franchise = ''
        
    if type(developers) == str:
        developers = developers.split(',')
    else: developers = ''
        
    if type(publishers) == str:
        publishers = publishers.split(',')
    else: publishers = ''
        
    if type(genres) == str:
        genres = genres.split(',')
    else: genres =''
        
    if type(online_mp) == str:
        if (online_mp == 'No Online Multiplayer') or (online_mp == '')or (online_mp == ' '):
            online_mp = False
        else:
            online_mp = True
    else: online_mp = False
            
    if type(offline_mp) == str:
        if (offline_mp == '1 Player') or (offline_mp == '') or (offline_mp == ' '):
            offline_mp = False
        else:
            offline_mp = True
    else: offline_mp = False
    
    game_features = {'genres':genres,
               'developers':developers,
               'publisher':publishers,
               'franchise':franchise,
               'online mp':online_mp,
               'offline mp':offline_mp}
    return(game_features)

def GetReccomendedGames(game_feat_dict, details_df):
    genres = game_feat_dict['genres']
    developers = game_feat_dict['developers']
    publishers = game_feat_dict['publisher']
    games_in_same_genre = {}
    #print('\nGAMES IN SAME GENRES:')
    for genre in genres:
        # Regex matchs either a comma following or the end of line so Action won't match Action Adventure
        games_in_same_genre[genre] =  details_df.loc[details_df['genre(s)'].str.contains(genre+'(,|$)')]
        #print(games_in_same_genre[genre]['name'],games_in_same_genre[genre]['genre(s)'])
        
    #print('\nGAMES BY SAME DEVELOPERS')
    games_by_same_dev = {}
    for dev in developers:
        games_by_same_dev[dev] =  details_df.loc[details_df['developer'].str.contains(dev+'(,|$)')]
        #print(games_by_same_dev[dev]['name'])

    #print('\nGAMES BY SAME PUBLISHERS')
    games_by_same_pub = {}
    for pub in publishers:
        games_by_same_pub[pub] =  details_df.loc[details_df['publisher']==pub]
        #print(games_by_same_pub[pub]['name'])
    
    return games_by_same_dev, games_by_same_pub, games_in_same_genre
    ## Missing online and offline multiplayer, and franchise
        
def MakeTopNLinks(source,values_df, N, already_included):
    already = already_included[:]
    topn_links = []
    i = 0
    names = list(values_df['name'])
    for name in names:
        if i >= N:
            break
        elif name in already:
            link_dict = {'source':source, 'target': name, 'strength': 0.001}
        else:
            link_dict = {'source':source, 'target': name, 'strength': 0.1}
            already.append(name)
            i += 1
        topn_links.append(link_dict)
    return topn_links, already

## Need to make a list of dictionaries for the links 
e.g. 
[{'source': 'AAL', 'target': 'AAR', 'strength': 0.1},

 {'source': 'AAL', 'target': 'BLL', 'strength': 0.3},
 
 {'source': 'AAL', 'target': 'CPH', 'strength': 0.2},
 
 {'source': 'AAL', 'target': 'LGW', 'strength': 0.1},]

In [98]:
a_game_name = 'Midnight Club: Los Angeles'
a_games_features = GetGameFeatures(a_game_name, ps3_details)
dev_games, pub_games, gen_games = GetReccomendedGames(gta_v_features, ps3_details)
already_list = [a_game_name]
n = 5
all_leaf_nodes = []
# Franchise then developer then publisher then genres then the two multiplayers

for key,val in gen_games.items():
    topn_by_gen, already_list = MakeTopNLinks(key, val, n, already_list)
    all_leaf_nodes += topn_by_gen
print(all_leaf_nodes)
# topn_by_action_adv, inc_act_adv = MakeTopNLinks('Action Adventure', games_in_same_genre['Action Adventure'], 10, inc_pub)
# print(inc_act_adv)
# print(topn_by_pub + topn_by_action_adv)

[{'source': 'Automobile', 'target': 'Need for Speed: Hot Pursuit', 'strength': 0.1}, {'source': 'Automobile', 'target': 'DiRT 2', 'strength': 0.1}, {'source': 'Automobile', 'target': 'GRID', 'strength': 0.1}, {'source': 'Automobile', 'target': 'Burnout Paradise', 'strength': 0.1}, {'source': 'Automobile', 'target': 'DiRT 3', 'strength': 0.1}, {'source': ' Driving', 'target': 'Wipeout HD Fury', 'strength': 0.1}, {'source': ' Driving', 'target': 'Need for Speed: Hot Pursuit', 'strength': 0.001}, {'source': ' Driving', 'target': 'DiRT 2', 'strength': 0.001}, {'source': ' Driving', 'target': 'Wipeout HD', 'strength': 0.1}, {'source': ' Driving', 'target': 'GRID', 'strength': 0.001}, {'source': ' Driving', 'target': 'Burnout Paradise', 'strength': 0.001}, {'source': ' Driving', 'target': 'DiRT 3', 'strength': 0.001}, {'source': ' Driving', 'target': 'Joe Danger', 'strength': 0.1}, {'source': ' Driving', 'target': 'Split/Second', 'strength': 0.1}, {'source': ' Driving', 'target': 'Need for S

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


## Need to make a list of dictionaries for the links 
e.g. 
[{'source': 'AAL', 'target': 'AAR', 'strength': 0.1},

 {'source': 'AAL', 'target': 'BLL', 'strength': 0.3},
 
 {'source': 'AAL', 'target': 'CPH', 'strength': 0.2},
 
 {'source': 'AAL', 'target': 'LGW', 'strength': 0.1},]

## Need to also make a list of dictionaries for nodes

In [108]:
def MakeNodes():
    pass
GTAV_ps3

,metascore,name,console,userscore,date,genre(s),developer,publisher,rating,esrb descriptors,number of online players,special controllers,number of players,official site,franchise
0,97,Grand Theft Auto V,PS3,8.3,"Sep 17, 2013","Action Adventure, Modern, Open-World",Rockstar North,Rockstar Games,M,,Up to 16 Players,,,http://www.rockstargames.com/,Grand Theft Auto


In [118]:
# Making them manually first as a test

gtav_root = {'id':'Grand Theft Auto V', 'group':1, 'label':'Grand Theft Auto V\n Rating: M\n, Metascore: 97\n, User score: 8.3\n', 'level':1}
other_game_nodes = []
for name in inc_act_adv:
    if name == 'Grand Theft Auto V':
        continue
    other_game_nodes.append({'id':name, 'group':1,'label':name,'level':3})
act_adv_node = {'id':'Action Adventure', 'group':3, 'label':'Action Adventure', 'level':2}
pub_node = {'id':'Rockstar Games', 'group':2, 'label':'Rockstar Games','level':2}

[gtav_root,act_adv_node, pub_node]+other_game_nodes

[{'id': 'Grand Theft Auto V',
  'group': 1,
  'label': 'Grand Theft Auto V\n Rating: M\n, Metascore: 97\n, User score: 8.3\n',
  'level': 1},
 {'id': 'Action Adventure',
  'group': 3,
  'label': 'Action Adventure',
  'level': 2},
 {'id': 'Rockstar Games', 'group': 2, 'label': 'Rockstar Games', 'level': 2},
 {'id': 'Grand Theft Auto IV',
  'group': 1,
  'label': 'Grand Theft Auto IV',
  'level': 3},
 {'id': 'Red Dead Redemption',
  'group': 1,
  'label': 'Red Dead Redemption',
  'level': 3},
 {'id': 'L.A. Noire', 'group': 1, 'label': 'L.A. Noire', 'level': 3},
 {'id': 'Max Payne 3', 'group': 1, 'label': 'Max Payne 3', 'level': 3},
 {'id': 'Red Dead Redemption: Undead Nightmare Pack',
  'group': 1,
  'label': 'Red Dead Redemption: Undead Nightmare Pack',
  'level': 3},
 {'id': 'Midnight Club: Los Angeles',
  'group': 1,
  'label': 'Midnight Club: Los Angeles',
  'level': 3},
 {'id': 'Red Dead Redemption: Liars and Cheats',
  'group': 1,
  'label': 'Red Dead Redemption: Liars and Cheats',